### Import des librairies et verification-GPU

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

!pip install opencv-python
# librairies générales
import pandas as pd
import cv2
import numpy as np
import tensorflow
# pour la data augmentation ,preproccesing et split
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
# pour les réseaux d'apprentissage
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping

print("Is GPU available for TensorFlow: ", tensorflow.config.list_physical_devices('GPU'))

2023-03-05 08:16:09.277765: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 08:16:11.389162: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 08:16:12.067018: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10784 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0001:00:00.0, compute capability: 3.7


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Is GPU available for TensorFlow:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Mise en place des données pour l'entrainement

##### Chargement de test csv/ modif pour charger les images du dataframe

In [3]:
df_train= pd.read_csv("data/train.csv")
df_train['path']=  df_train['image_id'].apply(lambda x: 'data/data/' + x)

# Partie I - Classification 2 à 2 (3 modeles)

### Etablissement des génerateurs pour l'entrainement et la validation

In [52]:
# inputs: Les images 232,232 seront redimmenssionner au format 256,256,1 et seront augmenter pour le generateur d'entrainement
# outputs: les labels seront rendu compatibles pour une loss: 'binary_crossentropy'  

class CustomDataGenerator(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256, 256), output_size=(256, 256), shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.output_size = output_size
        self.shuffle = shuffle
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_x = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), 1))
        
        for i, row in enumerate(batch_df.itertuples()):
            x = tensorflow.keras.utils.load_img(self.df["path"][i], target_size=(232,232), color_mode='grayscale') #chargement de l'image
            x = tensorflow.keras.preprocessing.image.img_to_array(x) # transformation en array numpy 
            x = self.augment_input(x)  # applique la data augmentation pour prévenir l'overfitting 
            x = x.astype(np.uint16)
            x = cv2.resize(x, dsize= self.output_size, interpolation=cv2.INTER_CUBIC) # redimmensionnement de l'image en 256*256
            x = np.reshape(x,(256,256,1)).astype('float32') / 255 # mise en forme et normalisation pour injection dans le CNN

            y= self.df["label"][i] # aller checher le label pour l'output
        
            
            batch_x[i] = x
            batch_y[i] = y
        
        return batch_x, batch_y
    
    def augment_input(self, x):
        # possible transformation sur l'image x choisit au hasard pour la data augmentation
        image_generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=20,
            zoom_range=0.1,   
            brightness_range=[0.9, 1.1],
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            horizontal_flip=True,
            fill_mode='reflect',
        )
        x = image_generator.random_transform(x)
        return x
    
    def on_epoch_end(self):
        # melange le dataframe à la fin de chaque epoch d'entrainement
        if self.shuffle:
            self.df = self.df.sample(frac=1)
            
            
class CustomDataGenerator_valid(Sequence):
    # similaire à CustomDataGenerator sans data augmentation pour la validation
    def __init__(self, df, batch_size, input_size=(256, 256), output_size=(256, 256), shuffle=False):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.output_size = output_size
        self.shuffle = shuffle
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_x = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), 1))
        
        for i, row in enumerate(batch_df.itertuples()):
            # for training
            x = tensorflow.keras.utils.load_img(self.df["path"][i], target_size=(232,232), color_mode='grayscale')
            x = tensorflow.keras.preprocessing.image.img_to_array(x)
            x = x.astype(np.uint16)
            x = cv2.resize(x, dsize= self.output_size, interpolation=cv2.INTER_CUBIC)
            x = np.reshape(x,(256,256,1)).astype('float32') / 255
            
            y= self.df["label"][i]
            
            batch_x[i] = x
            batch_y[i] = y
        
        return batch_x, batch_y

#### Etablissement des différents datasets pour classificateur 2 labels

In [6]:
df_train_np= df_train.drop(df_train[df_train['label'].str.contains('COVID')].index)  # df normal/pneumonie
df_train_nc= df_train.drop(df_train[df_train['label'].str.contains('PNEUMONIA')].index) # df normal/covid
df_train_pc=  df_train.drop(df_train[df_train['label'].str.contains('NORMAL')].index)  # df pneumonie/covid

df_train_np['label'] = df_train_np['label'].replace({'NORMAL': 0, 'PNEUMONIA': 1})
df_train_nc['label'] = df_train_nc['label'].replace({'NORMAL': 0, 'COVID': 1})
df_train_pc['label'] = df_train_pc['label'].replace({'PNEUMONIA': 0, 'COVID': 1})

## Partie I-1- Classificateur normal/pneumo

### Etablissement des différents datasets test/entrainement pour le classificateur normal/pneumo

In [7]:
df_np_train, df_np_test = train_test_split(df_train_np, test_size=0.2, random_state=168)
train_np_df, val_np_df = train_test_split(df_np_train, test_size=0.2, random_state= 761)
# Afficher le nombre de lignes de chaque ensemble de données
print('Nombre de lignes dans l\'ensemble d\'entraînement et de validation :', len(df_np_train))
print('Nombre de lignes dans l\'ensemble de test :', len(df_np_test))
print('Nombre de lignes dans l\'ensemble d\'entraînement :', len(train_np_df))
print('Nombre de lignes dans l\'ensemble de validation :', len(val_np_df))

Nombre de lignes dans l'ensemble d'entraînement et de validation : 2016
Nombre de lignes dans l'ensemble de test : 505
Nombre de lignes dans l'ensemble d'entraînement : 1612
Nombre de lignes dans l'ensemble de validation : 404


### Etablissement des génerateurs pour le classificateur normal/pneumo

In [53]:
# les index ont besoin d'etre réinitialiser pour éviter les erreurs lors de l'appel par le générateur
train_np_df=train_np_df.reset_index(drop= True)
val_np_df=val_np_df.reset_index(drop= True)
# les générateurs définissent le batch size ici fixé à 32
train_np_generator=CustomDataGenerator(train_np_df, batch_size=32)
valid_np_generator=CustomDataGenerator_valid(val_np_df, batch_size=32)

### Etablissement du modele et des hyperparametres pour le classificateur normal/pneumo

In [41]:
# parametre les dimensions des images 
img_width, img_height, channels = 256, 256, 1


# definition de la couche d'entrée
inputs = Input(shape=(img_width, img_height, channels))

# Blocs de convolution

conv1 = Conv2D(filters=8, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(inputs)
conv2 = Conv2D(filters=8, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv1)
pool1 = MaxPooling2D((2, 2))(conv2)

conv3 = Conv2D(filters=16, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(pool1)
conv4 = Conv2D(filters=16, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv3)
pool2 = MaxPooling2D((2, 2))(conv4)

conv5 = Conv2D(filters=32, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(pool2)
conv6 = Conv2D(filters=32, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv5)
pool3 = MaxPooling2D((2, 2))(conv6)

conv7 = Conv2D(filters=64, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(pool3)
conv8 = Conv2D(filters=64, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv7)
pool4 = MaxPooling2D((2, 2))(conv8)

conv9 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(pool4)
conv10 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv9)
conv11 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv10)
conv12 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv11)
pool5 = MaxPooling2D((2, 2))(conv12)


# definition du réseau 'fully connected'
flatten = Flatten()(pool5)
fc1 = Dense(512, activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.01))(flatten)
fc2 = Dense(256, activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.01))(fc1)
outputs = Dense(1, activation='sigmoid')(fc2)

# creation du modele
model_np = Model(inputs=inputs, outputs=outputs)

# creation de l'optimiseur
opt = Adam(learning_rate = 0.0001,epsilon=0.1)
# compilation du modele
model_np.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model_np.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 256, 256, 1)]     0         
                                                                 
 conv2d_84 (Conv2D)          (None, 256, 256, 8)       208       
                                                                 
 conv2d_85 (Conv2D)          (None, 256, 256, 8)       1608      
                                                                 
 max_pooling2d_35 (MaxPoolin  (None, 128, 128, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_86 (Conv2D)          (None, 128, 128, 16)      3216      
                                                                 
 conv2d_87 (Conv2D)          (None, 128, 128, 16)      2320      
                                                           

In [42]:
# Création d'un objet ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)
# Création d'un objet EarlyStopping
early_stop = EarlyStopping(monitor='accuracy', patience=8, verbose=1, restore_best_weights=True)

### Entrainement du modele de classification normal/pneumo

In [43]:
history_model_np = model_np.fit(train_np_generator, epochs=20, validation_data=valid_np_generator,callbacks=[reduce_lr,early_stop])

Epoch 1/20
51/51 [==============================] - 53s 927ms/step - loss: 31.0755 - accuracy: 0.5602 - val_loss: 31.0667 - val_accuracy: 0.5297 - lr: 1.0000e-04
Epoch 2/20
51/51 [==============================] - 48s 938ms/step - loss: 31.0533 - accuracy: 0.5620 - val_loss: 31.0394 - val_accuracy: 0.5297 - lr: 1.0000e-04
Epoch 3/20
51/51 [==============================] - 47s 914ms/step - loss: 31.0230 - accuracy: 0.5620 - val_loss: 31.0057 - val_accuracy: 0.5297 - lr: 1.0000e-04
Epoch 4/20
51/51 [==============================] - 48s 940ms/step - loss: 30.9909 - accuracy: 0.5682 - val_loss: 30.9632 - val_accuracy: 0.5594 - lr: 1.0000e-04
Epoch 5/20
51/51 [==============================] - 47s 925ms/step - loss: 30.9495 - accuracy: 0.5850 - val_loss: 30.9125 - val_accuracy: 0.6535 - lr: 1.0000e-04
Epoch 6/20
51/51 [==============================] - 46s 907ms/step - loss: 30.9012 - accuracy: 0.6452 - val_loss: 30.8555 - val_accuracy: 0.7178 - lr: 1.0000e-04
Epoch 7/20
51/51 [==========

### Calcul des prediction sur l'ensemble de test normal/pneumo

In [44]:
y_pred_liste=[]
y_true_liste=[]
y_pred_binary_liste=[]
df_np_test = df_np_test.reset_index(drop=True)

for i in range(len(df_np_test["path"])):
    x = tensorflow.keras.utils.load_img(df_np_test["path"][i], target_size=(232,232), color_mode='grayscale')
    x = tensorflow.keras.preprocessing.image.img_to_array(x)
    x = x.astype(np.uint16)
    x = cv2.resize(x, dsize= (256,256), interpolation=cv2.INTER_CUBIC)
    x = np.reshape(x,(256,256,1)).astype('float32') / 255
    x=np.resize(x, (1, *(256,256),1))
    y= df_np_test["label"][i]
    y= np.resize(y, (1, 1))
    y_pred = model_np.predict(x,verbose= 0)
    y_pred_binary = np.round(y_pred).astype(int)
    y_pred_liste.extend(y_pred)
    y_pred_binary_liste.extend(y_pred_binary)
    y_true_liste.extend(y)

In [45]:
y_pred_liste = np.reshape(y_pred_liste, (505))
y_true_liste = np.reshape(y_true_liste, (505))
y_pred_binary_liste = np.reshape(y_pred_binary_liste, (505))
print(pd.crosstab(y_true_liste, y_pred_binary_liste))

col_0    0    1
row_0          
0      237    7
1       58  203


### Sauvegarde du modele

In [46]:
model_np.save('clasificateur_normal_pneumo_2_87percent.h5')

## Partie I-2- Classificateur normal/covid

### Etablissement des différents datasets test/entrainement pour le classificateur normal/covid

In [48]:
df_nc_train, df_nc_test = train_test_split(df_train_nc, test_size=0.2, random_state=168)
train_nc_df, val_nc_df = train_test_split(df_nc_train, test_size=0.2, random_state= 761)
# Afficher le nombre de lignes de chaque ensemble de données
print('Nombre de lignes dans l\'ensemble d\'entraînement et de validation :', len(df_nc_train))
print('Nombre de lignes dans l\'ensemble de test :', len(df_nc_test))
print('Nombre de lignes dans l\'ensemble d\'entraînement :', len(train_nc_df))
print('Nombre de lignes dans l\'ensemble de validation :', len(val_nc_df))


Nombre de lignes dans l'ensemble d'entraînement et de validation : 1919
Nombre de lignes dans l'ensemble de test : 480
Nombre de lignes dans l'ensemble d'entraînement : 1535
Nombre de lignes dans l'ensemble de validation : 384


### Etablissement des génerateurs pour le classificateur normal/covid

In [54]:
train_nc_df=train_nc_df.reset_index(drop= True)
val_nc_df=val_nc_df.reset_index(drop= True)
train_nc_generator=CustomDataGenerator(train_nc_df, batch_size=32)
valid_nc_generator=CustomDataGenerator_valid(val_nc_df, batch_size=32)

### Etablissement du modele et des hyperparametres pour le classificateur normal/covid

In [63]:
# parametre les dimensions des images 
img_width, img_height, channels = 256, 256, 1


# definition de la couche d'entrée
inputs = Input(shape=(img_width, img_height, channels))

# Blocs de convolution

conv1 = Conv2D(filters=8, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(inputs)
conv2 = Conv2D(filters=8, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv1)
pool1 = MaxPooling2D((2, 2))(conv2)

conv3 = Conv2D(filters=16, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(pool1)
conv4 = Conv2D(filters=16, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv3)
pool2 = MaxPooling2D((2, 2))(conv4)

conv5 = Conv2D(filters=32, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(pool2)
conv6 = Conv2D(filters=32, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv5)
pool3 = MaxPooling2D((2, 2))(conv6)

conv7 = Conv2D(filters=64, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(pool3)
conv8 = Conv2D(filters=64, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv7)
pool4 = MaxPooling2D((2, 2))(conv8)

conv9 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(pool4)
conv10 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv9)
conv11 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv10)
conv12 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv11)
pool5 = MaxPooling2D((2, 2))(conv12)


# definition du réseau 'fully connected'
flatten = Flatten()(pool5)
fc1 = Dense(512, activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.01))(flatten)
fc2 = Dense(256, activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.01))(fc1)
outputs = Dense(1, activation='sigmoid')(fc2)

# creation du modele
model_nc = Model(inputs=inputs, outputs=outputs)

# creation de l'optimiseur
opt = Adam(learning_rate = 0.0001,epsilon=0.1)
# compilation du modele
model_nc.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model_nc.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 256, 256, 1)]     0         
                                                                 
 conv2d_108 (Conv2D)         (None, 256, 256, 8)       208       
                                                                 
 conv2d_109 (Conv2D)         (None, 256, 256, 8)       1608      
                                                                 
 max_pooling2d_45 (MaxPoolin  (None, 128, 128, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_110 (Conv2D)         (None, 128, 128, 16)      3216      
                                                                 
 conv2d_111 (Conv2D)         (None, 128, 128, 16)      2320      
                                                           

 ### Entrainement du modele de classification normal/covid

In [64]:
history_model_nc = model_nc.fit(train_nc_generator, epochs=20, validation_data=valid_nc_generator,callbacks=[reduce_lr,early_stop])

Epoch 1/20
48/48 [==============================] - 50s 952ms/step - loss: 31.0367 - accuracy: 0.6313 - val_loss: 30.8993 - val_accuracy: 0.7500 - lr: 1.0000e-04
Epoch 2/20
48/48 [==============================] - 46s 967ms/step - loss: 30.8610 - accuracy: 0.8182 - val_loss: 30.6882 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 3/20
48/48 [==============================] - 45s 933ms/step - loss: 30.7251 - accuracy: 0.8684 - val_loss: 30.5324 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 4/20
48/48 [==============================] - 45s 945ms/step - loss: 30.6221 - accuracy: 0.8925 - val_loss: 30.4311 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 5/20
48/48 [==============================] - 46s 963ms/step - loss: 30.5464 - accuracy: 0.9147 - val_loss: 30.3754 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 6/20
48/48 [==============================] - 45s 944ms/step - loss: 30.5058 - accuracy: 0.9127 - val_loss: 30.3275 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 7/20
48/48 [==========

### Calcul des prediction sur l'ensemble de test normal/covid

In [65]:
df_nc_test = df_nc_test.reset_index(drop=True)
y_pred_liste_nc=[]
y_true_liste_nc=[]
y_pred_binary_liste_nc=[]
df_nc_test = df_nc_test.reset_index(drop=True)

for i in range(len(df_nc_test["path"])):
    x = tensorflow.keras.utils.load_img(df_nc_test["path"][i], target_size=(232,232), color_mode='grayscale')
    x = tensorflow.keras.preprocessing.image.img_to_array(x)
    x = x.astype(np.uint16)
    x = cv2.resize(x, dsize= (256,256), interpolation=cv2.INTER_CUBIC)
    x = np.reshape(x,(256,256,1)).astype('float32') / 255
    x=np.resize(x, (1, *(256,256),1))
    y= df_nc_test["label"][i]
    y= np.resize(y, (1, 1))
    y_pred = model_nc.predict(x,verbose= 0)
    y_pred_binary = np.round(y_pred).astype(int)
    y_pred_liste_nc.extend(y_pred)
    y_pred_binary_liste_nc.extend(y_pred_binary)
    y_true_liste_nc.extend(y)

In [66]:
y_pred_liste_nc = np.reshape(y_pred_liste_nc, (480))
y_true_liste_nc = np.reshape(y_true_liste_nc, (480))
y_pred_binary_liste_nc = np.reshape(y_pred_binary_liste_nc, (480))
print(pd.crosstab(y_true_liste_nc, y_pred_binary_liste_nc))

col_0    0    1
row_0          
0      237    5
1       16  222


### Sauvegarde du modele

In [67]:
model_nc.save('clasificateur_normal_covid_2_95percent.h5')

## Partie I-3- Classificateur normal/covid

### Etablissement des différents datasets test/entrainement pour le classificateur pneumo/covid

In [68]:
df_pc_train, df_pc_test = train_test_split(df_train_pc, test_size=0.2, random_state=168)
train_pc_df, val_pc_df = train_test_split(df_pc_train, test_size=0.2, random_state= 761)
# Afficher le nombre de lignes de chaque ensemble de données
print('Nombre de lignes dans l\'ensemble d\'entraînement et de validation :', len(df_pc_train))
print('Nombre de lignes dans l\'ensemble de test :', len(df_pc_test))
print('Nombre de lignes dans l\'ensemble d\'entraînement :', len(train_pc_df))
print('Nombre de lignes dans l\'ensemble de validation :', len(val_pc_df))

Nombre de lignes dans l'ensemble d'entraînement et de validation : 1918
Nombre de lignes dans l'ensemble de test : 480
Nombre de lignes dans l'ensemble d'entraînement : 1534
Nombre de lignes dans l'ensemble de validation : 384


### Etablissement des génerateurs pour le classificateur pneumo/covid

In [69]:
train_pc_df=train_pc_df.reset_index(drop= True)
val_pc_df=val_pc_df.reset_index(drop= True)
train_pc_generator=CustomDataGenerator(train_pc_df, batch_size=32)
valid_pc_generator=CustomDataGenerator_valid_test(val_pc_df, batch_size=32)

### Etablissement du modele et des hyperparametres pour le classificateur normal/covid

In [70]:
# parametre les dimensions des images 
img_width, img_height, channels = 256, 256, 1


# definition de la couche d'entrée
inputs = Input(shape=(img_width, img_height, channels))

# Blocs de convolution

conv1 = Conv2D(filters=8, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(inputs)
conv2 = Conv2D(filters=8, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv1)
pool1 = MaxPooling2D((2, 2))(conv2)

conv3 = Conv2D(filters=16, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(pool1)
conv4 = Conv2D(filters=16, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv3)
pool2 = MaxPooling2D((2, 2))(conv4)

conv5 = Conv2D(filters=32, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(pool2)
conv6 = Conv2D(filters=32, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv5)
pool3 = MaxPooling2D((2, 2))(conv6)

conv7 = Conv2D(filters=64, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(pool3)
conv8 = Conv2D(filters=64, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv7)
pool4 = MaxPooling2D((2, 2))(conv8)

conv9 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(pool4)
conv10 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv9)
conv11 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv10)
conv12 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(conv11)
pool5 = MaxPooling2D((2, 2))(conv12)


# definition du réseau 'fully connected'
flatten = Flatten()(pool5)
fc1 = Dense(512, activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.01))(flatten)
fc2 = Dense(256, activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.01))(fc1)
outputs = Dense(1, activation='sigmoid')(fc2)

# creation du modele
model_pc = Model(inputs=inputs, outputs=outputs)

# creation de l'optimiseur
opt = Adam(learning_rate = 0.0001,epsilon=0.1)
# compilation du modele
model_pc.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model_pc.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 256, 256, 1)]     0         
                                                                 
 conv2d_120 (Conv2D)         (None, 256, 256, 8)       208       
                                                                 
 conv2d_121 (Conv2D)         (None, 256, 256, 8)       1608      
                                                                 
 max_pooling2d_50 (MaxPoolin  (None, 128, 128, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_122 (Conv2D)         (None, 128, 128, 16)      3216      
                                                                 
 conv2d_123 (Conv2D)         (None, 128, 128, 16)      2320      
                                                          

### Entrainement du modele de classification pneumo/covid

In [71]:
history_model_pc = model_pc.fit(train_pc_generator, epochs=20, validation_data=valid_pc_generator,callbacks=[reduce_lr,early_stop])

Epoch 1/20
48/48 [==============================] - 50s 941ms/step - loss: 31.0692 - accuracy: 0.5007 - val_loss: 31.0646 - val_accuracy: 0.5312 - lr: 1.0000e-04
Epoch 2/20
48/48 [==============================] - 44s 916ms/step - loss: 31.0548 - accuracy: 0.5007 - val_loss: 31.0478 - val_accuracy: 0.5312 - lr: 1.0000e-04
Epoch 3/20
48/48 [==============================] - 44s 910ms/step - loss: 31.0354 - accuracy: 0.5013 - val_loss: 31.0265 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 4/20
48/48 [==============================] - 44s 924ms/step - loss: 31.0123 - accuracy: 0.5111 - val_loss: 31.0015 - val_accuracy: 0.5312 - lr: 1.0000e-04
Epoch 5/20
48/48 [==============================] - 44s 915ms/step - loss: 30.9859 - accuracy: 0.5574 - val_loss: 30.9725 - val_accuracy: 0.6562 - lr: 1.0000e-04
Epoch 6/20
48/48 [==============================] - 43s 902ms/step - loss: 30.9546 - accuracy: 0.6258 - val_loss: 30.9395 - val_accuracy: 0.6875 - lr: 1.0000e-04
Epoch 7/20
48/48 [==========

### Calcul des prediction sur l'ensemble de test pneumo/covid

In [73]:
y_pred_liste_pc=[]
y_true_liste_pc=[]
y_pred_binary_liste_pc=[]
df_pc_test = df_pc_test.reset_index(drop=True)

for i in range(len(df_pc_test["path"])):
    x = tensorflow.keras.utils.load_img(df_pc_test["path"][i], target_size=(232,232), color_mode='grayscale')
    x = tensorflow.keras.preprocessing.image.img_to_array(x)
    x = x.astype(np.uint16)
    x = cv2.resize(x, dsize= (256,256), interpolation=cv2.INTER_CUBIC)
    x = np.reshape(x,(256,256,1)).astype('float32') / 255
    x=np.resize(x, (1, *(256,256),1))
    y= df_pc_test["label"][i]
    y= np.resize(y, (1, 1))
    y_pred = model_pc.predict(x,verbose= 0)
    y_pred_binary = np.round(y_pred).astype(int)
    y_pred_liste_pc.extend(y_pred)
    y_pred_binary_liste_pc.extend(y_pred_binary)
    y_true_liste_pc.extend(y)

In [74]:
y_pred_liste_pc = np.reshape(y_pred_liste_pc, (480))
y_true_liste_pc = np.reshape(y_true_liste_pc, (480))
y_pred_binary_liste_pc = np.reshape(y_pred_binary_liste_pc, (480))
print(pd.crosstab(y_true_liste_pc, y_pred_binary_liste_pc))

col_0    0    1
row_0          
0      233   11
1       15  221


### Sauvegarde du modele

In [75]:
model_pc.save('clasificateur_pneumo_covid_2_94percent.h5')

## Partie II- Classificateur pour les trois classes(Class3) normal/pneumo/covid par palier

### Etablissement des différents datasets test/entrainement pour le classificateur 3 classes

In [79]:
df_train['label'] = df_train['label'].replace({'NORMAL': 0, 'PNEUMONIA': 1, 'COVID': 2})

df_all_train, df_all_test = train_test_split(df_train, test_size=0.2, random_state=168)
train_all_df, val_all_df = train_test_split(df_all_train, test_size=0.2, random_state= 761)
# Afficher le nombre de lignes de chaque ensemble de données
print('Nombre de lignes dans l\'ensemble d\'entraînement et de validation :', len(df_all_train))
print('Nombre de lignes dans l\'ensemble de test :', len(df_all_test))
print('Nombre de lignes dans l\'ensemble d\'entraînement :', len(train_all_df))
print('Nombre de lignes dans l\'ensemble de validation :', len(val_all_df))

Nombre de lignes dans l'ensemble d'entraînement et de validation : 2927
Nombre de lignes dans l'ensemble de test : 732
Nombre de lignes dans l'ensemble d'entraînement : 2341
Nombre de lignes dans l'ensemble de validation : 586


### Etablissement des génerateurs pour l'entrainement et la validation du class3

In [81]:
class CustomDataGenerator_class3(Sequence):
    # similaire au générateur d'avant sauf qu'on labelle les outputs en one-hot encoder de facon à pouvoir l'utiliser avec un softmax et une loss categorical crossentropy
    # Il sort également 4 inputs: 1 pour chaque classificateur 2-2 + 1 pour la colonne vertébrale de ce nouveau classificateur
    def __init__(self, df, batch_size, input_size=(256, 256), output_size=(256, 256), shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.output_size = output_size
        self.shuffle = shuffle
        self.num_classes = 3 
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_x = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))
        
        for i, row in enumerate(batch_df.itertuples()):
            x = tensorflow.keras.utils.load_img(self.df["path"][i], target_size=(232,232), color_mode='grayscale')
            x = tensorflow.keras.preprocessing.image.img_to_array(x)
            x = self.augment_input(x)
            x = x.astype(np.uint16)
            x = cv2.resize(x, dsize= self.output_size, interpolation=cv2.INTER_CUBIC)
            x = np.reshape(x,(256,256,1)).astype('float32') / 255
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
            
            batch_x[i] = x
            batch_y[i] = y
        
        return (batch_x,batch_x,batch_x,batch_x), batch_y
    
    def augment_input(self, x):
        image_generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=20, 
            zoom_range=0.1,     
            brightness_range=[0.9, 1.1],
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            horizontal_flip=True,
            fill_mode='reflect',
        )
        x = image_generator.random_transform(x)
        return x
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1)
            
class CustomDataGenerator_valid_class3(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256, 256), output_size=(256, 256), shuffle=False):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.output_size = output_size
        self.shuffle = shuffle
        self.num_classes = 3 
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_x = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))
        
        for i, row in enumerate(batch_df.itertuples()):
            x = tensorflow.keras.utils.load_img(self.df["path"][i], target_size=(232,232), color_mode='grayscale')
            x = tensorflow.keras.preprocessing.image.img_to_array(x)
            x = x.astype(np.uint16)
            x = cv2.resize(x, dsize= self.output_size, interpolation=cv2.INTER_CUBIC)
            x = np.reshape(x,(256,256,1)).astype('float32') / 255
            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
            
            batch_x[i] = x
            batch_y[i] = y
        
        return (batch_x, batch_x, batch_x,batch_x), batch_y

In [82]:
train_all_df = train_all_df.reset_index(drop= True)
val_all_df = val_all_df.reset_index(drop= True)

train_all_generator_glob=CustomDataGenerator_class3(train_all_df, batch_size=32)
valid_all_generator_glob=CustomDataGenerator_valid_class3(val_all_df, batch_size=32)

### Appel des classificateurs 2-2

In [84]:
model_nc = tensorflow.keras.models.load_model('clasificateur_normal_covid_2_95percent.h5',compile=False)
model_pc = tensorflow.keras.models.load_model('clasificateur_pneumo_covid_2_94percent.h5',compile=False)
model_np = tensorflow.keras.models.load_model('clasificateur_normal_pneumo_2_87percent.h5',compile=False)

### Etablissement du modele et des hyperparametres pour le classificateur Class3

In [109]:
from tensorflow.keras.layers import concatenate

# Freeze the weights of the encoder layers
for layer in model_nc.layers[:-2]:
    layer.trainable = False
for layer in model_pc.layers[:-2]:
    layer.trainable = False
for layer in model_np.layers[:-2]:
    layer.trainable = False

new_inputs = Input(shape=(img_width, img_height, channels))

nc_dense_1 = model_nc.layers[1].output
pc_dense_1 = model_pc.layers[1].output
np_dense_1 = model_np.layers[1].output

nc_dense_2 = model_nc.layers[2].output
pc_dense_2 = model_pc.layers[2].output
np_dense_2 = model_np.layers[2].output

nc_dense_3 = model_nc.layers[4].output
pc_dense_3 = model_pc.layers[4].output
np_dense_3 = model_np.layers[4].output

nc_dense_4 = model_nc.layers[5].output
pc_dense_4 = model_pc.layers[5].output
np_dense_4 = model_np.layers[5].output


nc_dense_5 = model_nc.layers[7].output
pc_dense_5 = model_pc.layers[7].output
np_dense_5 = model_np.layers[7].output

nc_dense_6 = model_nc.layers[8].output
pc_dense_6 = model_pc.layers[8].output
np_dense_6 = model_np.layers[8].output

nc_dense_7 = model_nc.layers[10].output
pc_dense_7 = model_pc.layers[10].output
np_dense_7 = model_np.layers[10].output

nc_dense_8 = model_nc.layers[11].output
pc_dense_8 = model_pc.layers[11].output
np_dense_8 = model_np.layers[11].output

nc_dense_9 = model_nc.layers[13].output
pc_dense_9 = model_pc.layers[13].output
np_dense_9 = model_np.layers[13].output

nc_dense_10= model_nc.layers[14].output
pc_dense_10= model_pc.layers[14].output
np_dense_10= model_np.layers[14].output

nc_dense_11= model_nc.layers[15].output
pc_dense_11= model_pc.layers[15].output
np_dense_11= model_np.layers[15].output

nc_dense_12= model_nc.layers[16].output
pc_dense_12= model_pc.layers[16].output
np_dense_12= model_np.layers[16].output

nc_dense_13= model_nc.layers[-3].output
pc_dense_13= model_pc.layers[-3].output
np_dense_13= model_np.layers[-3].output

nc_dense_14= model_nc.layers[-2].output
pc_dense_14= model_pc.layers[-2].output
np_dense_14= model_np.layers[-2].output

# parametre les dimensions des images 
img_width, img_height, channels = 256, 256, 1


# definition de la couche d'entrée
new_inputs = Input(shape=(img_width, img_height, channels))
# Blocs de convolution

conv1 = Conv2D(filters=8, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(new_inputs)
conv1 = concatenate([conv1,nc_dense_1,pc_dense_1,np_dense_1])
conv2 = Conv2D(filters=8, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(conv1)
conv2 = concatenate([conv2,nc_dense_2,pc_dense_2,np_dense_2])
conv2 = Conv2D(filters=8, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(conv2)
pool1 = MaxPooling2D((2, 2))(conv2)

conv3 = Conv2D(filters=16, kernel_size=5, activation='relu', padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(pool1)
conv3 = concatenate([conv3,nc_dense_3,pc_dense_3,np_dense_3])
conv4 = Conv2D(filters=16, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(conv3)
conv4 = concatenate([conv4,nc_dense_4,pc_dense_4,np_dense_4])
conv4 = Conv2D(filters=16, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(conv4)
pool2 = MaxPooling2D((2, 2))(conv4)

conv5 = Conv2D(filters=32, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(pool2)
conv5 = concatenate([conv5,nc_dense_5,pc_dense_5,np_dense_5])
conv6 = Conv2D(filters=32, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(conv5)
conv6 = concatenate([conv6,nc_dense_6,pc_dense_6,np_dense_6])
conv6 = Conv2D(filters=32, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(conv6)
pool3 = MaxPooling2D((2, 2))(conv6)

conv7 = Conv2D(filters=64, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(pool3)
conv7 = concatenate([conv7,nc_dense_7,pc_dense_7,np_dense_7])
conv8 = Conv2D(filters=64, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(conv7)
conv8 = concatenate([conv8,nc_dense_8,pc_dense_8,np_dense_8])
conv8 = Conv2D(filters=64, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(conv8)
pool4 = MaxPooling2D((2, 2))(conv8)

conv9 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(pool4)
conv9 = concatenate([conv9,nc_dense_9,pc_dense_9,np_dense_9])
conv10 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(conv9)
conv10 = concatenate([conv10,nc_dense_10,pc_dense_10,np_dense_10])
conv11 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(conv10)
conv11 = concatenate([conv11,nc_dense_11,pc_dense_11,np_dense_11])
conv12 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(conv11)
conv12 = concatenate([conv12,nc_dense_12,pc_dense_12,np_dense_12])
conv12 = Conv2D(filters=128, kernel_size=3, activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(conv12)
pool5 = MaxPooling2D((2, 2))(conv12)

flatten = Flatten()(pool5)
fc1 = Dense(512, activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.1))(flatten)
fc1 = concatenate([fc1,nc_dense_13,pc_dense_13,np_dense_13])
fc1 = Dense(512, activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.1))(fc1)
fc2 = Dense(256, activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.1))(fc1)
fc2 = concatenate([fc2,nc_dense_14,pc_dense_14,np_dense_14])
fc2 = Dense(256, activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.1))(fc2)
outputs = Dense(3, activation='softmax')(fc2)

model_Class3 = Model(inputs=[new_inputs,model_nc.input, model_pc.input, model_np.input], outputs=outputs)

# creation de l'optimiseur
optimizer = Adam(learning_rate = 0.0001,epsilon=0.1)

model_Class3.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model_Class3.summary()

Model: "model_24"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_34 (InputLayer)          [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_10 (InputLayer)          [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_11 (InputLayer)          [(None, 256, 256, 1  0           []                               
                                )]                                                         

In [110]:
# Création d'un objet ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)
# Création d'un objet EarlyStopping
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)

In [111]:
history_model_Class3 = model_Class3.fit(train_all_generator_glob, epochs=35, validation_data=valid_all_generator_glob,callbacks=[reduce_lr,early_stop])

Epoch 1/35
74/74 [==============================] - 77s 909ms/step - loss: 595.2174 - accuracy: 0.5399 - val_loss: 592.6113 - val_accuracy: 0.6246 - lr: 1.0000e-04
Epoch 2/35
74/74 [==============================] - 67s 892ms/step - loss: 590.5267 - accuracy: 0.6929 - val_loss: 587.9716 - val_accuracy: 0.7184 - lr: 1.0000e-04
Epoch 3/35
74/74 [==============================] - 67s 903ms/step - loss: 585.2432 - accuracy: 0.7211 - val_loss: 582.1378 - val_accuracy: 0.7509 - lr: 1.0000e-04
Epoch 4/35
74/74 [==============================] - 68s 910ms/step - loss: 579.0479 - accuracy: 0.7433 - val_loss: 575.5369 - val_accuracy: 0.7474 - lr: 1.0000e-04
Epoch 5/35
74/74 [==============================] - 69s 925ms/step - loss: 572.1606 - accuracy: 0.7651 - val_loss: 568.3231 - val_accuracy: 0.8123 - lr: 1.0000e-04
Epoch 6/35
74/74 [==============================] - 70s 937ms/step - loss: 564.6922 - accuracy: 0.7873 - val_loss: 560.6522 - val_accuracy: 0.8140 - lr: 1.0000e-04
Epoch 7/35
74/74

### Calcul des prediction sur l'ensemble de test dataframe

In [112]:
y_pred_liste=[]
y_true_liste=[]
y_pred_argmax_liste=[]
df_all_test = df_all_test.reset_index(drop=True)

for i in range(len(df_all_test["path"])):
    x = tensorflow.keras.utils.load_img(df_all_test["path"][i], target_size=(232,232), color_mode='grayscale')
    x = tensorflow.keras.preprocessing.image.img_to_array(x)
    x = x.astype(np.uint16)
    x = cv2.resize(x, dsize= (256,256), interpolation=cv2.INTER_CUBIC)
    x = np.reshape(x,(256,256,1)).astype('float32') / 255
    x=np.resize(x, (1, *(256,256),1))
    y= df_all_test["label"][i]
    y= np.resize(y, (1, 1))
    y_pred = model_Class3.predict((x,x,x,x),verbose= 0)
    y_pred_argmax = np.argmax(y_pred, axis=1)
    y_pred_liste.extend(y_pred)
    y_pred_argmax_liste.extend(y_pred_argmax)
    y_true_liste.extend(y)

In [113]:
y_pred_liste = np.reshape(y_pred_liste, (732,3))
y_true_liste = np.reshape(y_true_liste, (732))
y_pred_argmax_liste = np.reshape(y_pred_argmax_liste, (732))
print(pd.crosstab(y_true_liste, y_pred_argmax_liste))


col_0    0    1    2
row_0               
0      184   72    4
1       21  221    5
2       10   12  203


### Sauvegarde du modele 

In [115]:
model_Class3.save('clasificateur_full_2_83percent.h5')

#### relancer pour voir si amelioration

In [116]:
history_model_Class3 = model_Class3.fit(train_all_generator_glob, epochs=15, validation_data=valid_all_generator_glob,callbacks=[reduce_lr,early_stop])

Epoch 1/15
74/74 [==============================] - 67s 897ms/step - loss: 452.3416 - accuracy: 0.8505 - val_loss: 447.6040 - val_accuracy: 0.8447 - lr: 1.0000e-04
Epoch 2/15
74/74 [==============================] - 67s 893ms/step - loss: 443.0376 - accuracy: 0.8633 - val_loss: 438.3801 - val_accuracy: 0.8754 - lr: 1.0000e-04
Epoch 3/15
74/74 [==============================] - 67s 904ms/step - loss: 434.0306 - accuracy: 0.8466 - val_loss: 429.4365 - val_accuracy: 0.9061 - lr: 1.0000e-04
Epoch 4/15
74/74 [==============================] - 70s 941ms/step - loss: 425.2217 - accuracy: 0.8599 - val_loss: 420.7262 - val_accuracy: 0.8754 - lr: 1.0000e-04
Epoch 5/15
74/74 [==============================] - 70s 937ms/step - loss: 416.6115 - accuracy: 0.8582 - val_loss: 412.1649 - val_accuracy: 0.8754 - lr: 1.0000e-04
Epoch 6/15
74/74 [==============================] - 69s 931ms/step - loss: 408.0992 - accuracy: 0.8625 - val_loss: 403.8061 - val_accuracy: 0.9061 - lr: 1.0000e-04
Epoch 7/15
74/74

In [119]:
y_pred_liste=[]
y_true_liste=[]
y_pred_argmax_liste=[]
df_all_test = df_all_test.reset_index(drop=True)

for i in range(len(df_all_test["path"])):
    x = tensorflow.keras.utils.load_img(df_all_test["path"][i], target_size=(232,232), color_mode='grayscale')
    x = tensorflow.keras.preprocessing.image.img_to_array(x)
    x = x.astype(np.uint16)
    x = cv2.resize(x, dsize= (256,256), interpolation=cv2.INTER_CUBIC)
    x = np.reshape(x,(256,256,1)).astype('float32') / 255
    x=np.resize(x, (1, *(256,256),1))
    y= df_all_test["label"][i]
    y= np.resize(y, (1, 1))
    y_pred = model_Class3.predict((x,x,x,x),verbose= 0)
    y_pred_argmax = np.argmax(y_pred, axis=1)
    y_pred_liste.extend(y_pred)
    y_pred_argmax_liste.extend(y_pred_argmax)
    y_true_liste.extend(y)

In [120]:
y_pred_liste = np.reshape(y_pred_liste, (732,3))
y_true_liste = np.reshape(y_true_liste, (732))
y_pred_argmax_liste = np.reshape(y_pred_argmax_liste, (732))
print(pd.crosstab(y_true_liste, y_pred_argmax_liste))


col_0    0    1    2
row_0               
0      205   49    6
1       28  210    9
2       10    9  206


### legere amelioration donc sauvegarde 

In [121]:
model_Class3.save('clasificateur_full_2_84percent.h5')

### calcul des predictions pour soumissions

In [122]:
df_test= pd.read_csv("data/test.csv")
df_test['path']=  df_test['image_id'].apply(lambda x: 'data/data/' + x)


In [123]:
y_pred_liste=[]
y_true_liste=[]
y_pred_argmax_liste=[]

for i in range(len(df_test["path"])):
    x = tensorflow.keras.utils.load_img(df_test["path"][i], target_size=(232,232), color_mode='grayscale')
    x = tensorflow.keras.preprocessing.image.img_to_array(x)
    x = x.astype(np.uint16)
    x = cv2.resize(x, dsize= (256,256), interpolation=cv2.INTER_CUBIC)
    x = np.reshape(x,(256,256,1)).astype('float32') / 255
    x = np.resize(x, (1, *(256,256),1))
    y_pred = model_Class3.predict((x,x,x,x),verbose= 0)
    y_pred_argmax = np.argmax(y_pred, axis=1)
    y_pred_liste.extend(y_pred)
    y_pred_argmax_liste.extend(y_pred_argmax)

In [124]:
y_pred_argmax_liste = np.reshape(y_pred_argmax_liste, (1569))
df_test["label"]= y_pred_argmax_liste
df_test['label'] = df_test['label'].replace({0 : 'NORMAL', 1:'PNEUMONIA', 2 :'COVID'})
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1569 entries, 0 to 1568
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_id    1569 non-null   object
 1   trustii_id  1569 non-null   int64 
 2   path        1569 non-null   object
 3   label       1569 non-null   object
dtypes: int64(1), object(3)
memory usage: 49.2+ KB


In [125]:
df_test.to_csv('submission_clasificateur_palier_1_84percent.csv', index=False, columns=['image_id','trustii_id','label'], encoding='UTF-8')